In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv(r"..\imputed_data.csv")

In [4]:
df.head(5)

,surrogate_key,creation_full_date,year,month,quarter,day_of_week,month_name,day_name,date_key,tag,score,view_count,favorite_count,comment_count,answer_count,answer_status
0,-2937206958259993047,2020-05-01 00:00:00.000,2020,5,2,6,May,Friday,20200501,python,0,47,NaN,0,1,answered
1,2708228319434441337,2020-05-03 00:00:00.000,2020,5,2,1,May,Sunday,20200503,python,0,34,NaN,0,1,answered
2,2708228319434441337,2020-05-03 00:00:00.000,2020,5,2,1,May,Sunday,20200503,python,0,73,NaN,0,1,answered
3,-7128293146829867177,2020-05-04 00:00:00.000,2020,5,2,2,May,Monday,20200504,python,0,405,NaN,0,1,answered
4,-7128293146829867177,2020-05-04 00:00:00.000,2020,5,2,2,May,Monday,20200504,python,0,58,NaN,0,1,answered


In [10]:
df.columns

Index(['surrogate_key', 'creation_full_date', 'year', 'month', 'quarter',
       'day_of_week', 'month_name', 'day_name', 'date_key', 'tag', 'score',
       'view_count', 'favorite_count', 'comment_count', 'answer_count',
       'answer_status'],
      dtype='object')

In [7]:
df_original = df

# STEP 1: First Normal Form (1NF)

In [11]:
df_1nf = df_original.copy()

# STEP 2: Second Normal Form (2NF)

In [12]:
# Identify: surrogate_key is the primary key
# All date-related fields depend on date, not on surrogate_key
# Split into separate tables

# Table 1: Questions (main entity)
questions = df_original[['surrogate_key', 'creation_full_date', 'year', 'month', 'quarter',
       'day_of_week', 'month_name', 'day_name', 'date_key', 'tag', 'score',
       'view_count', 'favorite_count', 'comment_count', 'answer_count',
       'answer_status']].copy()

# Table 2: Date Dimension (date attributes)
date_dim = df_original[[
    'date_key', 'creation_full_date', 'year', 'month', 
    'quarter', 'day_of_week', 'month_name', 'day_name'
]].drop_duplicates().copy()

# STEP 3: Third Normal Form (3NF)

In [16]:

# ============================================================================
# STEP 3: Third Normal Form (3NF)
# ============================================================================
print("=" * 80)
print("STEP 3: THIRD NORMAL FORM (3NF)")
print("=" * 80)
print("Requirements: In 2NF + No transitive dependencies")
print()

# Identify transitive dependencies:
# 1. In Date_Dimension: year, month → quarter (transitive)
# 2. In Date_Dimension: day_of_week → day_name (transitive)
# 3. In Date_Dimension: month → month_name (transitive)

# Final normalized tables:

# Table 1: Questions (Fact Table)
tbl_questions = questions[[
    'surrogate_key', 'date_key', 'tag', 'score',
    'view_count', 'favorite_count', 'comment_count',
    'answer_count', 'answer_status'
]].copy()

print("Table 1: Questions (Fact Table)")
print("-" * 80)
print(tbl_questions)
print("\nPrimary Key: surrogate_key")
print("Foreign Key: date_key → Date_Dimension")
print()

# Table 2: Date_Dimension (minimal, no transitive deps)
tbl_date_dimension = date_dim[[
    'date_key', 'creation_full_date', 'year', 
    'month', 'day_of_week'
]].copy()

print("Table 2: Date_Dimension")
print("-" * 80)
print(tbl_date_dimension)
print("\nPrimary Key: date_key")
print("Foreign Keys: month → Month_Lookup, day_of_week → Day_Lookup")
print()

# Table 3: Month_Lookup (reference table)
month_data = date_dim[['month', 'month_name']].drop_duplicates().sort_values('month')
# Add quarter calculation
month_data['quarter'] = ((month_data['month'] - 1) // 3) + 1
tbl_month_lookup = month_data.copy()

print("Table 3: Month_Lookup")
print("-" * 80)
print(tbl_month_lookup)
print("\nPrimary Key: month")
print()

# Table 4: Day_Lookup (reference table)
tbl_day_lookup = date_dim[['day_of_week', 'day_name']].drop_duplicates().sort_values('day_of_week')

print("Table 4: Day_Lookup")
print("-" * 80)
print(tbl_day_lookup)
print("\nPrimary Key: day_of_week")
print()

STEP 3: THIRD NORMAL FORM (3NF)
Requirements: In 2NF + No transitive dependencies

Table 1: Questions (Fact Table)
--------------------------------------------------------------------------------
               surrogate_key  date_key         tag  score  view_count  \
0       -2937206958259993047  20200501      python      0          47   
1        2708228319434441337  20200503      python      0          34   
2        2708228319434441337  20200503      python      0          73   
3       -7128293146829867177  20200504      python      0         405   
4       -7128293146829867177  20200504      python      0          58   
...                      ...       ...         ...    ...         ...   
4718834  5024919420517454387  20191214        dart     37       23021   
4718835 -3458375839532563935  20200214     reactjs     33       52954   
4718836   202256289945761400  20200221     combine     10        6938   
4718837  1021510682880289207  20201125  kubernetes     78       70153   
4

In [17]:
tbl_questions.head()

,surrogate_key,date_key,tag,score,view_count,favorite_count,comment_count,answer_count,answer_status
0,-2937206958259993047,20200501,python,0,47,NaN,0,1,answered
1,2708228319434441337,20200503,python,0,34,NaN,0,1,answered
2,2708228319434441337,20200503,python,0,73,NaN,0,1,answered
3,-7128293146829867177,20200504,python,0,405,NaN,0,1,answered
4,-7128293146829867177,20200504,python,0,58,NaN,0,1,answered


In [18]:
tbl_date_dimension.head()

,date_key,creation_full_date,year,month,day_of_week
0,20200501,2020-05-01 00:00:00.000,2020,5,6
1,20200503,2020-05-03 00:00:00.000,2020,5,1
3,20200504,2020-05-04 00:00:00.000,2020,5,2
6,20200505,2020-05-05 00:00:00.000,2020,5,3
8,20200506,2020-05-06 00:00:00.000,2020,5,4


In [19]:
tbl_month_lookup.head()

,month,month_name,quarter
1932,1,January,1
2049,2,February,1
2190,3,March,1
255,4,April,2
0,5,May,2


In [22]:
tbl_day_lookup.head()

,day_of_week,day_name
1,1,Sunday
3,2,Monday
6,3,Tuesday
8,4,Wednesday
24,5,Thursday


In [28]:
## SAVE the tables

tbl_questions.to_csv(r"..\normaliazed_dataset\tbl_fact_questions.csv", index=False)
tbl_date_dimension.to_csv(r"..\normaliazed_dataset\tbl_date_dimension.csv", index=False)
tbl_month_lookup.to_csv(r"..\normaliazed_dataset\tbl_month_lookup.csv", index=False)
tbl_day_lookup.to_csv(r"..\normaliazed_dataset\tbl_day_lookup.csv", index=False)

# Summary

3NF FINAL SCHEMA:

1. Questions (Fact Table)
   - surrogate_key (PK)
   - date_key (FK)
   - tag
   - score
   - review_count
   - favorite_count
   - comment_count
   - answer_count
   - answer_status

2. Date_Dimension
   - date_key (PK)
   - creation_full_date
   - year
   - month (FK)
   - day_of_week (FK)

3. Month_Lookup
   - month (PK)
   - month_name
   - quarter

4. Day_Lookup
   - day_of_week (PK)
   - day_name